<h5><b>Install Package and Library</h5></b>

In [4]:
# Install package
!pip install selenium
!pip install webdriver-manager

# Install library
import datetime as dt
import numpy as np
import os
import pandas as pd
import time
from datetime import date, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By

# Install wget
!apt-get install wget

# Download the chromedriver.exe from GitHub
os.system('wget https://github.com/SamapanThongmee/Thai-Stock-Tickers-from-web-scrapping/blob/main/chromedriver.exe')

# Create function
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()
# driver = webdriver.Chrome('./chromedriver')

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


<h4><b>Scrap Tickers</b></h4>

In [114]:
def get_stock_data(url):
    driver = web_driver()
    driver.get(url)
    data = driver.page_source
    data_df = pd.read_html(data)[2]

    clean_column = [c.replace('  (Click to sort Ascending)', '') for c in data_df.columns]
    data_df.columns = clean_column
    data_df = data_df.set_index('Symbol')

    stocks = []
    for stock in data_df.index:
        stock = stock.split()[0]
        stocks.append(stock)
    return stocks

# Define URLs for different categories
categories = {
    'AGRO_AGRI': 'https://www.set.or.th/en/market/index/set/agro/agri',
    'AGRO_FOOD': 'https://www.set.or.th/en/market/index/set/agro/food',
    'CONSUMP_FASION': 'https://www.set.or.th/en/market/index/set/consump/fashion',
    'CONSUMP_HOME': 'https://www.set.or.th/en/market/index/set/consump/home',
    'CONSUMP_PERSON': 'https://www.set.or.th/en/market/index/set/consump/person',
    'FINCIAL_BANK': 'https://www.set.or.th/en/market/index/set/fincial/bank',
    'FINCIAL_FIN': 'https://www.set.or.th/en/market/index/set/fincial/fin',
    'FINCIAL_INSUR': 'https://www.set.or.th/en/market/index/set/fincial/insur',
    'INDUS_AUTO' : 'https://www.set.or.th/en/market/index/set/indus/auto',
    'INDUS_IMM' : 'https://www.set.or.th/en/market/index/set/indus/imm',
    'INDUS_PAPER' : 'https://www.set.or.th/en/market/index/set/indus/paper',
    'INDUS_PETRO' : 'https://www.set.or.th/en/market/index/set/indus/petro',
    'INDUS_PKG' : 'https://www.set.or.th/en/market/index/set/indus/pkg',
    'INDUS_STEEL' : 'https://www.set.or.th/en/market/index/set/indus/steel',
    'PROPCON_CONMAT' : 'https://www.set.or.th/en/market/index/set/propcon/conmat',
    'PROPCON_CONS' : 'https://www.set.or.th/en/market/index/set/propcon/cons',
    'PROPCON_PROP' : 'https://www.set.or.th/en/market/index/set/propcon/prop',
    'PROPCON_PF_REIT' : 'https://www.set.or.th/en/market/index/set/propcon/pf&reit',
    'RESOURC_ENERG' : 'https://www.set.or.th/en/market/index/set/resourc/energ',
    'SERVICE_COMM' : 'https://www.set.or.th/en/market/index/set/service/comm',
    'SERVICE_HELTH' : 'https://www.set.or.th/en/market/index/set/service/helth',
    'SERVICE_MEDIA' : 'https://www.set.or.th/en/market/index/set/service/media',
    'SERVICE_PROF' : 'https://www.set.or.th/en/market/index/set/service/prof',
    'SERVICE_TOURISM' : 'https://www.set.or.th/en/market/index/set/service/tourism',
    'SERVICE_TRANS' : 'https://www.set.or.th/en/market/index/set/service/trans',
    'TECH_ETRON' : 'https://www.set.or.th/en/market/index/set/tech/etron',
    'TECH_ICT' : 'https://www.set.or.th/en/market/index/set/tech/ict'
}

result = {}
for category, url in categories.items():
    stocks = get_stock_data(url)
    result[category] = stocks

import pandas as pd

data_list = []
for i in range(27):  # Assuming you have 27 elements in result
    sector = list(result.keys())[i]
    ticker = list(result.values())[i]
    data = {'Sector': [sector] * len(ticker), 'Ticker': ticker}
    data_list.append(pd.DataFrame(data))

data = pd.concat(data_list, axis=0).reset_index(drop=True)
data[['Sector', 'Industry']] = data['Sector'].str.split('_', 1, expand=True)
tickers = data[['Sector', 'Ticker']]
tickers

C:\Users\samap\AppData\Local\Temp\ipykernel_184\2868581475.py:63: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  data[['Sector', 'Industry']] = data['Sector'].str.split('_', 1, expand=True)


,Sector,Ticker
0,AGRO,EE
1,AGRO,GFPT
2,AGRO,LEE
3,AGRO,MAX
4,AGRO,NER
...,...,...
687,TECH,SYNEX
688,TECH,THCOM
689,TECH,TKC
690,TECH,TRUE


<h4><b>Export to .csv</b></h4>

In [115]:
tickers.to_csv('tickers.csv')